In [1]:
import os
import openai

In [2]:
!pip install openai

In [3]:
!pip install praw

In [4]:
import praw

In [5]:
%env

{'TERM_PROGRAM': 'Apple_Terminal',
 'TERM': 'xterm-color',
 'SHELL': '/bin/zsh',
 'TMPDIR': '/var/folders/7d/zmvddj4x3_g38kcblzy5bqkw0000gn/T/',
 'CONDA_SHLVL': '1',
 'TERM_PROGRAM_VERSION': '447',
 'CONDA_PROMPT_MODIFIER': '(base) ',
 'TERM_SESSION_ID': '1DFA3523-2A5C-4705-80E2-44F553524DCB',
 'USER': 'thomasthephasdin',
 'CONDA_EXE': '/Users/thomasthephasdin/opt/anaconda3/bin/conda',
 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.QtZwSvivfO/Listeners',
 '_CE_CONDA': '',
 'PATH': '/opt/local/bin:/opt/local/sbin:/Users/thomasthephasdin/opt/anaconda3/bin:/Users/thomasthephasdin/opt/anaconda3/condabin:/Library/Frameworks/Python.framework/Versions/3.10/bin:/Library/Frameworks/Python.framework/Versions/3.10/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/Apple/usr/bin',
 'LaunchInstanceID': '7FC143E8-EC7E-4675-8132-945B5B919408',
 '__CFBundleIdentifier': 'com.apple.Terminal',
 'CONDA_PREFIX': '/Users/thomasthephasdin/opt/anaconda3',
 'PWD': '/User

In [8]:
import os

In [35]:
openai.api_key = os.getenv('OPENAI_API_KEY') 

In [37]:
%env

{'TERM_PROGRAM': 'Apple_Terminal',
 'TERM': 'xterm-color',
 'SHELL': '/bin/zsh',
 'TMPDIR': '/var/folders/7d/zmvddj4x3_g38kcblzy5bqkw0000gn/T/',
 'CONDA_SHLVL': '1',
 'TERM_PROGRAM_VERSION': '447',
 'CONDA_PROMPT_MODIFIER': '(base) ',
 'TERM_SESSION_ID': '1DFA3523-2A5C-4705-80E2-44F553524DCB',
 'USER': 'thomasthephasdin',
 'CONDA_EXE': '/Users/thomasthephasdin/opt/anaconda3/bin/conda',
 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.QtZwSvivfO/Listeners',
 '_CE_CONDA': '',
 'PATH': '/opt/local/bin:/opt/local/sbin:/Users/thomasthephasdin/opt/anaconda3/bin:/Users/thomasthephasdin/opt/anaconda3/condabin:/Library/Frameworks/Python.framework/Versions/3.10/bin:/Library/Frameworks/Python.framework/Versions/3.10/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/Apple/usr/bin',
 'LaunchInstanceID': '7FC143E8-EC7E-4675-8132-945B5B919408',
 '__CFBundleIdentifier': 'com.apple.Terminal',
 'CONDA_PREFIX': '/Users/thomasthephasdin/opt/anaconda3',
 'PWD': '/User

In [12]:
reddit = praw.Reddit(client_id=os.getenv('client_id'),
                    client_secret=os.getenv('client_secret'),
                    user_agent = "sentiment analysis test")

In [16]:
subreddit_stocks = reddit.subreddit('stocks')

In [19]:
subreddit_stocks.accounts_active

5775

In [24]:
for post in subreddit_stocks.hot(limit=5):
    print(post.title)
    submission = reddit.submission(post.id)
    
    counter = 0
    for comment in submission.comments:
        print(comment.body)
        counter+=1
        if counter == 2:
            break

Rate My Portfolio - r/Stocks Quarterly Thread March 2023
I am 26, so I have several decades until retirement. My goal is to hold the stocks in my portfolio perpetually, though I'm not reserved to make adjustments and sell when a) I see an opportunity to take profits b) my thesis changes. My long term goal is passive income generation, though I don't chase yields. I don't pay much attention to valuation or macro, as I think that since I plan on holding these stocks for decades while buying every month, those don't matter to me much. I invest regularly every month, so I'm okay with buying at a higher valuation at times and lower valuation at other times. In my opinion, buying at the best possible time is more difficult than buying a good company. That said, if I see an undervalued stock I like, I don't hesitate to buy more and when I make my regular contributions, I try weigh towards stocks that might have been beaten down or seem undervalued. My last review in one of these was [Dec 2021

In [39]:
def get_titles_and_comments(subreddit='stocks',limit=6,num_comments=3,skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    title_and_comments = {}
    
    for counter,post in enumerate(subreddit.hot(limit=limit)):
        
        if counter < skip_first:
            continue
        counter += (1-skip_first)
        
        title_and_comments[counter] = ""
        submission = reddit.submission(post.id)
        title = post.title
        
        title_and_comments[counter] +='Title: '+title+"\n\n"
        title_and_comments[counter] += "Comments: \n\n"
        
        comment_counter = 0
        for comment in submission.comments:
            if not comment.body == "[deleted]":
                title_and_comments[counter] += comment.body+"\n"
                comment_counter += 1
            if comment_counter == num_comments:
                break
    return title_and_comments

In [40]:
titles_and_comments = get_titles_and_comments()


In [41]:
titles_and_comments

{1: 'Title: Alphanso Stock Research: Why BioLife Solutions, Inc. ($BLFS) Stock Has a Score of 9.7/10\n\nComments: \n\n',
 2: 'Title: BriaCell Therapeutics Corp. (Nasdaq: BCTX, BCTXW) will be sharing "refreshed survival updates, new patient data, and potential late-breaking discoveries."\n\nComments: \n\n',
 3: 'Title: Future Plans of Water On Demand™ $OCLN\n\nComments: \n\n',
 4: 'Title: $META stock\n\nComments: \n\n'}

In [42]:
def create_prompt(title_and_comments):
    task = "Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive, negative, or neutral. If no ticker or company is mentioned write 'No company mentioned'\n\n"
    return task+title_and_comments

In [43]:
print(create_prompt(titles_and_comments[1]))

Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive, negative, or neutral. If no ticker or company is mentioned write 'No company mentioned'

Title: Alphanso Stock Research: Why BioLife Solutions, Inc. ($BLFS) Stock Has a Score of 9.7/10

Comments: 




In [44]:
for key, title_with_coments in  titles_and_comments.items():
    
    prompt = create_prompt(title_with_coments)
    
    response = openai.Completion.create(engine='text-davinci-003',
                                       prompt=prompt,
                                       max_tokens=256,
                                       temperature=0,
                                       top_p=1.0)
    print(title_with_coments)
    print(f"Sentiment Report from OpenAI: {response['choices'][0]['text']}")
    print('---------------------')

Title: Alphanso Stock Research: Why BioLife Solutions, Inc. ($BLFS) Stock Has a Score of 9.7/10

Comments: 


Sentiment Report from OpenAI: "BioLife Solutions, Inc. ($BLFS) is a great stock to invest in. It has a strong balance sheet and a great outlook for the future."

Company: BioLife Solutions, Inc. ($BLFS)
Sentiment: Positive
---------------------
Title: BriaCell Therapeutics Corp. (Nasdaq: BCTX, BCTXW) will be sharing "refreshed survival updates, new patient data, and potential late-breaking discoveries."

Comments: 


Sentiment Report from OpenAI: This is great news for BriaCell Therapeutics Corp.

Ticker: BCTX, BCTXW
Sentiment: Positive
---------------------
Title: Future Plans of Water On Demand™ $OCLN

Comments: 


Sentiment Report from OpenAI: The company is looking to expand its operations to new markets

Ticker: $OCLN
Sentiment: Neutral
---------------------
Title: $META stock

Comments: 


Sentiment Report from OpenAI: META has been on a steady rise since the start of the